In [506]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from datetime import datetime


df_raw = pd.read_csv("HouseData.csv")
display(df_raw.head())

df = df_raw.copy()


,Unnamed: 0,district,price,address,AdUpdateDate,Category,GrossSquareMeters,BuildingAge,NumberFloorsofBuilding,UsingStatus,...,PriceStatus,RentalIncome,NumberOfBalconies,BalconyType,HallSquareMeters,WCSquareMeters,IsItVideoNavigable?,Subscription,BathroomSquareMeters,BalconySquareMeters
0,0,adalar,"3,100,000TL","['Anasayfa', 'Satılık Daire', 'İstanbul Satılı...",24 Şubat 2022,Satılık,160 m2,21 Ve Üzeri,3,Mülk Sahibi Oturuyor,...,Genel Fiyat,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,adalar,"1,600,000TL","['Anasayfa', 'Satılık Daire', 'İstanbul Satılı...",02 Mart 2022,Satılık,120 m2,5-10,3,Mülk Sahibi Oturuyor,...,Genel Fiyat,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,adalar,"18,500,000TL","['Anasayfa', 'Satılık Müstakil Ev', 'İstanbul ...",11 Şubat 2022,Satılık,350 m2,21 Ve Üzeri,2,Mülk Sahibi Oturuyor,...,Genel Fiyat,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,adalar,"9,500,000TL","['Anasayfa', 'Satılık Bina', 'İstanbul Satılık...",11 Şubat 2022,Satılık,550 m2,11-15,3,Mülk Sahibi Oturuyor,...,Genel Fiyat,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,adalar,"25,000,000TL","['Anasayfa', 'Satılık Köşk', 'İstanbul Satılık...",19 Ocak 2022,Satılık,840 m2,21 Ve Üzeri,4,Boş,...,Genel Fiyat,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [507]:
df = df.drop(columns=['Unnamed: 0', 'address', 'AdUpdateDate', 'IsItVideoNavigable?', 'Subscription', 'PriceStatus', 'BalconyType', 'HallSquareMeters', 'WCSquareMeters', 'BathroomSquareMeters', 'BalconySquareMeters', 'RentalIncome', 'NumberOfBalconies', "Category", "Type"], errors='ignore')
df['GrossSquareMeters'] = df['GrossSquareMeters'].astype(str).str.replace(r'\s*m2', '', regex=True).astype(float)

df['price'] = df['price'].astype(str).str.replace(r'\D', '', regex=True).astype(float)
df['NetSquareMeters'] = df['NetSquareMeters'].astype(str).str.replace(r'\D', '', regex=True).astype(int)
df['NetSquareMeters'] = ( df['NetSquareMeters'] - 2 ) / 10
df['NetSquareMeters'] = df['NetSquareMeters'].astype(int)
# Fiyatı olmayan (NaN) satırları veri setinden çıkarıyoruz, çünkü hedef değişkenimiz bu.
df.dropna(subset=['price'], inplace=True)

print(df["BuildingAge"].unique())
age_mapping = {
    '0 (Yeni)': 0,    # Yeni eklenen değer.
    '1': 1,
    '2': 2,
    '3': 3,
    '4': 4,
    '5-10': 7.5,      # Aralığın ortası (5+10)/2
    '11-15': 13,      # Ortalaması
    '16-20': 18,      # Ortalaması
    '20 Ve Üzeri': 20, # Sadece yazı kısmı çıkarıldı
    '21 Ve Üzeri': 21  # Sadece yazı kısmı çıkarıldı
}
df['BuildingAge'] = df['BuildingAge'].astype(str).str.strip().replace(age_mapping)
# Eşleşmeyen veya dönüştürülemeyen değerleri sayısal olmayan (NaN) olarak işaretle.
df['BuildingAge'] = pd.to_numeric(df['BuildingAge'], errors='coerce')
# Kalan NaN değerleri (eğer varsa) sütunun medyanı ile doldur.
df['BuildingAge'] = df['BuildingAge'].fillna(df['BuildingAge'].median())

print(df["NumberFloorsofBuilding"].isnull().sum())
# Binanın kat sayısını sayısal değere dönüştürme
df['NumberFloorsofBuilding'] = pd.to_numeric(df['NumberFloorsofBuilding'], errors='coerce')

district_freq = df['district'].value_counts(normalize=True) # Yüzde olarak frekans
df['district_encoded'] = df['district'].map(district_freq)
df = df.drop(columns=['district']) # Orijinal district sütununu çıkarıyoruz

df = pd.get_dummies(df, columns=["UsingStatus"], dummy_na=False, drop_first=False, dtype=int)

for col in df.select_dtypes(include=np.number).columns:
    if df[col].isnull().any():
        if col != 'price':
            df[col] = df[col].fillna(df[col].median())
    print(f"{col}: {df[col].isnull().sum()}")

df.loc[df["EligibilityForInvestment"] == "Bilinmiyor", "EligibilityForInvestment"] = None
false_drops = ['EligibilityForInvestment', 'BuildStatus', 'TitleStatus', 'NumberOfWCs', 'StructureType']

for col in df.select_dtypes(include=['object', 'category']).columns:
    if df[col].isnull().any():
        if not df[col].mode().empty:
            fill_value = df[col].mode()[0]
            df[col] = df[col].fillna(fill_value)
            if col not in false_drops:
                df = pd.get_dummies(df, columns=[col], dummy_na=False, drop_first=True,dtype=int)

            else:
                df = pd.get_dummies(df, columns=[col], dummy_na=False, drop_first=False,dtype=int)

df = pd.get_dummies(df, columns=["HeatingType","CreditEligibility","InsideTheSite"], dummy_na=False, drop_first=False, dtype=int)

loc_freq = df['FloorLocation'].value_counts(normalize=True) # Yüzde olarak frekans
df['floor_encoded'] = df['FloorLocation'].map(loc_freq)
df = df.drop(columns=['FloorLocation']) 

month_map_for_conversion = {'Ocak': '01', 'Şubat': '02', 'Mart': '03', 'Nisan': '04', 'Mayıs': '05', 'Haziran': '06','Temmuz': '07', 'Ağustos': '08', 'Eylül': '09', 'Ekim': '10', 'Kasım': '11', 'Aralık': '12'}
temp_date_series = df['AdCreationDate'].astype(str)
for month_tr, month_num_str in month_map_for_conversion.items():
    temp_date_series = temp_date_series.str.replace(month_tr, month_num_str, regex=False)

# Dönüştürülmüş tarih serisini 'dd mm yyyy' formatında olacak şekilde düzenliyoruz
df['AdCreationDate_dt_temp'] = pd.to_datetime(temp_date_series, format='%d %m %Y', errors='coerce')
df['Ad_Yil'] = df['AdCreationDate_dt_temp'].dt.year
df['Ad_Ay'] = df['AdCreationDate_dt_temp'].dt.month
df['Ad_TotalGun'] = df['AdCreationDate_dt_temp'].dt.dayofyear
df = df.drop(columns=['AdCreationDate', 'AdCreationDate_dt_temp'])
# 'AdCreationDate' sütununu datetime formatına dönüştürdükten sonra, yıl, ay ve gün bilgilerini ayrı sütunlara ekledik.

temp_rooms_series = df['NumberOfRooms'].astype(str).str.lower().str.strip()
temp_rooms_series = temp_rooms_series.str.replace('stüdyo', '1', regex=False)
list_of_num_strings_series = temp_rooms_series.str.findall(r'(\d+\.?\d*)')
df['TotalRooms'] = list_of_num_strings_series.apply(lambda num_list: sum([float(s) for s in num_list]) if num_list else np.nan)
df = df.drop(columns=['NumberOfRooms'])
# NumberOfBathrooms sütununu işleme 

df['NumberOfBathrooms'] = df['NumberOfBathrooms'].replace({"Yok":"0"}, regex=False)
df['NumberOfBathrooms'] = df['NumberOfBathrooms'].str.replace(r'(\d+)\s*\+\s*$', r'\1', regex=True)
df['NumberOfBathrooms'] = pd.to_numeric(df['NumberOfBathrooms'], errors='coerce')


display(df)
display(df.dtypes)

['21 Ve Üzeri' '5-10' '11-15' '2' '0 (Yeni)' '3' '4' '16-20' '1'
 '20 Ve Üzeri']
0
price: 0
GrossSquareMeters: 0
BuildingAge: 0
NumberFloorsofBuilding: 0
NetSquareMeters: 0
district_encoded: 0
UsingStatus_Boş: 0
UsingStatus_Kiracı Oturuyor: 0
UsingStatus_Mülk Sahibi Oturuyor: 0


C:\Users\emirc\AppData\Local\Temp\ipykernel_9372\4284894483.py:24: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['BuildingAge'] = df['BuildingAge'].astype(str).str.strip().replace(age_mapping)


,price,GrossSquareMeters,BuildingAge,NumberFloorsofBuilding,NumberOfBathrooms,NetSquareMeters,district_encoded,UsingStatus_Boş,UsingStatus_Kiracı Oturuyor,UsingStatus_Mülk Sahibi Oturuyor,...,CreditEligibility_Bilinmiyor,CreditEligibility_Krediye Uygun,CreditEligibility_Krediye Uygun Değil,InsideTheSite_Evet,InsideTheSite_Hayır,floor_encoded,Ad_Yil,Ad_Ay,Ad_TotalGun,TotalRooms
0,3100000.0,160.0,21.0,3,2,120,0.001034,0,0,1,...,0,1,0,0,1,0.011767,2022,2,55,4.0
1,1600000.0,120.0,7.5,3,1,100,0.001034,0,0,1,...,0,1,0,0,1,0.072590,2022,2,52,3.0
2,18500000.0,350.0,21.0,2,3,300,0.001034,0,0,1,...,0,1,0,0,1,0.075253,2022,2,42,4.0
3,9500000.0,550.0,13.0,3,4,540,0.001034,0,0,1,...,0,0,1,0,1,0.075253,2022,2,34,8.0
4,25000000.0,840.0,21.0,4,3,700,0.001034,1,0,0,...,0,1,0,0,1,0.075253,2022,1,19,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25150,1850000.0,180.0,7.5,4,2,160,0.007593,0,0,1,...,0,1,0,0,1,0.103160,2021,2,36,5.0
25151,1100000.0,52.0,3.0,16,1,35,0.007593,0,1,0,...,0,1,0,0,1,0.112065,2020,6,171,2.0
25152,9000000.0,176.0,0.0,25,2,175,0.007593,1,0,0,...,0,1,0,0,1,0.003220,2019,12,364,4.0
25153,13000000.0,170.0,7.5,30,2,155,0.007593,1,0,0,...,0,1,0,0,1,0.003021,2019,2,47,4.0


price                     float64
GrossSquareMeters         float64
BuildingAge               float64
NumberFloorsofBuilding      int64
NumberOfBathrooms           int64
                           ...   
floor_encoded             float64
Ad_Yil                      int32
Ad_Ay                       int32
Ad_TotalGun                 int32
TotalRooms                float64
Length: 64, dtype: object

In [508]:
numerical_cols_for_winsorize = [
    'price', 'GrossSquareMeters', 'NetSquareMeters',
    'NumberOfBathrooms', 'NumberOfRooms', 'NumberOfWCs',
    'FloorLocation', 'NumberOfBalconies', 'HallSquareMeters',
    'district_encoded', 'floor_encoded', 'NumberFloorsofBuilding',
    'UsingStatus_Kullanım Durumu Kiralık', 'UsingStatus_Kullanım Durumu Satılık',
    'UsingStatus_Kullanım Durumu Satılık Kiralık', 'UsingStatus_Kullanım Durumu Diğer',
]
# Winsorization işlemi için gerekli sütunları belirledik.
# Bu sütunlar, veri setindeki aşırı uç değerleri sınırlandırmak için kullanılacaktır.
for col in numerical_cols_for_winsorize:
    if col in df.columns and df[col].isnull().sum() < len(df[col]): 
        non_nan_values = df[col].dropna()
        if not non_nan_values.empty:
            lower_bound = df[col].quantile(0.01)
            upper_bound = df[col].quantile(0.99)
            if pd.notna(lower_bound) and pd.notna(upper_bound) and lower_bound < upper_bound :
                df[col] = df[col].clip(lower=lower_bound, upper=upper_bound)
                print(f"'{col}' sütunu %1 ve %99 persentilleri arasında sınırlandı.")


'price' sütunu %1 ve %99 persentilleri arasında sınırlandı.
'GrossSquareMeters' sütunu %1 ve %99 persentilleri arasında sınırlandı.
'NetSquareMeters' sütunu %1 ve %99 persentilleri arasında sınırlandı.
'NumberOfBathrooms' sütunu %1 ve %99 persentilleri arasında sınırlandı.
'district_encoded' sütunu %1 ve %99 persentilleri arasında sınırlandı.
'floor_encoded' sütunu %1 ve %99 persentilleri arasında sınırlandı.
'NumberFloorsofBuilding' sütunu %1 ve %99 persentilleri arasında sınırlandı.


In [509]:
y_log = np.log1p(df['price'])  # Log dönüşümü yapıyoruz

X = df.drop(columns=['price'])
y = df['price']

X_train_orig, X_test_orig, y_train_orig, y_test_orig = train_test_split(X, y, test_size=0.2, random_state=42)

# Temel Karar Ağacı Modeli
tree_reg_base = DecisionTreeRegressor(random_state=42)
tree_reg_base.fit(X_train_orig, y_train_orig)
y_pred_tree_base = tree_reg_base.predict(X_test_orig)
mse_tree_base = mean_squared_error(y_test_orig, y_pred_tree_base)
r2_tree_base = r2_score(y_test_orig, y_pred_tree_base)
print(f"Temel Karar Ağacı Modeli - MSE: {mse_tree_base:.4f}, R²: {r2_tree_base:.4f}")



Temel Karar Ağacı Modeli - MSE: 478561368894209.7500, R²: -0.5957


In [510]:
X_train, X_test, y_train, y_test = train_test_split(X, y_log, test_size=0.2, random_state=42)

# Log dönüşümü uygulandıktan sonra, eğitim ve test setlerini oluşturuyoruz ve modelimizi oluşturuyoruz.
tree_reg_base = DecisionTreeRegressor(random_state=42)
tree_reg_base.fit(X_train, y_train)
y_pred_tree_base = tree_reg_base.predict(X_test)
mse_tree_base = mean_squared_error(y_test, y_pred_tree_base)
r2_tree_base = r2_score(y_test, y_pred_tree_base)
print(f"Temel Karar Ağacı Modeli - MSE: {mse_tree_base:.4f}, R²: {r2_tree_base:.4f}")

Temel Karar Ağacı Modeli - MSE: 1.2147, R²: 0.1704


In [511]:
# Karar Ağacı Modeli için Grid Search ile Hiperparametre Optimizasyonu
param_grid_tree = {"max_depth": [5, 10, None], "min_samples_split": [2, 5, 10], "min_samples_leaf": [1, 2, 5]}
grid_search_tree = GridSearchCV(DecisionTreeRegressor(random_state=42), param_grid_tree, cv=3, scoring="neg_mean_squared_error", n_jobs=-1)
grid_search_tree.fit(X_train, y_train)
best_tree_model = grid_search_tree.best_estimator_
best_tree_params = grid_search_tree.best_params_
print("En iyi parametreler (Karar Ağacı):", best_tree_params)
y_pred_best_tree = best_tree_model.predict(X_test) 
mse_best_tree = mean_squared_error(y_test, y_pred_best_tree)
r2_best_tree = r2_score(y_test, y_pred_best_tree)
print(f"Optimize Edilmiş Karar Ağacı - MSE: {mse_best_tree:.4f}, R²: {r2_best_tree:.4f}")

En iyi parametreler (Karar Ağacı): {'max_depth': 10, 'min_samples_leaf': 5, 'min_samples_split': 2}
Optimize Edilmiş Karar Ağacı - MSE: 0.7283, R²: 0.5026


In [512]:
X_train_fe = X_train.copy() 
X_test_fe = X_test.copy()   

X_train_fe['sqm_x_age'] = X_train_fe['GrossSquareMeters'] * X_train_fe['BuildingAge']
X_test_fe['sqm_x_age'] = X_test_fe['GrossSquareMeters'] * X_test_fe['BuildingAge'] # Özellik mühendisliği ile yeni bir özellik oluşturuyoruz.
median_sqm_x_age_train = X_train_fe['sqm_x_age'].median()
X_train_fe['sqm_x_age'] = X_train_fe['sqm_x_age'].fillna(0)
X_test_fe['sqm_x_age'] = X_test_fe['sqm_x_age'].fillna(0)

# Özellik mühendisliği ile yeni bir özellik oluşturulduktan sonra, modelimizi yeniden oluşturuyoruz.
tree_reg_fe = DecisionTreeRegressor(**best_tree_params, random_state=42)
tree_reg_fe.fit(X_train_fe, y_train)
y_pred_tree_fe = tree_reg_fe.predict(X_test_fe)
mse_tree_fe = mean_squared_error(y_test, y_pred_tree_fe)
r2_tree_fe = r2_score(y_test, y_pred_tree_fe)
print(f"Özellik Mühendisliği ile Karar Ağacı - MSE: {mse_tree_fe:.4f}, R²: {r2_tree_fe:.4f}")

Özellik Mühendisliği ile Karar Ağacı - MSE: 0.7273, R²: 0.5032


In [513]:
print("\n--- Özellik Seçimi (Iterative Removal) Başlıyor ---")
features_to_keep = X_train_fe.columns.tolist()
if not features_to_keep:
     X_train_selected_fe = X_train_fe.copy() 
     X_test_selected_fe = X_test_fe.copy()
     mse_selected_fe = np.nan 
     r2_selected_fe = np.nan
else:
    selection_model_base = DecisionTreeRegressor(**best_tree_params, random_state=42)
    selection_model_base.fit(X_train_fe, y_train)
    current_best_r2 = r2_score(y_test, selection_model_base.predict(X_test_fe[X_train_fe.columns]))
    print(f"Özellik seçimi için başlangıç R² (tüm X_train_fe ile): {current_best_r2:.4f}")

    removed_features_count = 0
    initial_features_list = X_train_fe.columns.tolist()

    for feature_to_test in initial_features_list:
        if feature_to_test not in features_to_keep: 
            continue
        if len(features_to_keep) <= 1: 
            print("En az bir özellik kalmalı, daha fazla çıkarma yapılamaz.")
            break

        temp_features = features_to_keep.copy()
        temp_features.remove(feature_to_test)
        
        if not temp_features: 
            print(f"'{feature_to_test}' çıkarılırsa özellik kalmıyor, atlanıyor.")
            continue

        selection_model_iter = DecisionTreeRegressor(**best_tree_params, random_state=42)
        selection_model_iter.fit(X_train_fe[temp_features], y_train)
        r2_temp = r2_score(y_test, selection_model_iter.predict(X_test_fe[temp_features]))

        if r2_temp > current_best_r2:
            previous_r2 = current_best_r2
            features_to_keep = temp_features 
            current_best_r2 = r2_temp 
            removed_features_count += 1
            print(f"'{feature_to_test}' çıkarıldı. Yeni R²: {current_best_r2:.4f} (Artış: {current_best_r2 - previous_r2:.4f})")
        
    print(f"--- Özellik Seçimi Tamamlandı. {removed_features_count} özellik çıkarıldı. Kalan özellik sayısı: {len(features_to_keep)} ---")
    print(f"Seçilen özellikler: {features_to_keep}")

    X_train_selected_fe = X_train_fe[features_to_keep]
    X_test_selected_fe = X_test_fe[features_to_keep]
    
    print("\n--- Seçilmiş Özellikler ile Karar Ağacı Modeli ---")
    tree_reg_selected_fe = DecisionTreeRegressor(**best_tree_params, random_state=42)
    tree_reg_selected_fe.fit(X_train_selected_fe, y_train) 
    y_pred_log_selected_fe = tree_reg_selected_fe.predict(X_test_selected_fe) 
    if y_test_orig is not None:
        y_pred_orig_selected_fe = np.expm1(y_pred_log_selected_fe)
        mse_selected_fe = mean_squared_error(y_test_orig, y_pred_orig_selected_fe)
    else:
        mse_selected_fe = np.nan
    r2_selected_fe = r2_score(y_test, y_pred_log_selected_fe)
    print(f"Seçilmiş Özellikler ile Karar Ağacı - MSE (orijinal): {mse_selected_fe:.4f}, R² (log): {r2_selected_fe:.4f}")

# Burada sütunlar tek tek çıkarılacak ve her seferinde modelin performansı değerlendiriliyor, model performansı iyileşirse o sütun çıkarılıyor.


--- Özellik Seçimi (Iterative Removal) Başlıyor ---
Özellik seçimi için başlangıç R² (tüm X_train_fe ile): 0.5032
'GrossSquareMeters' çıkarıldı. Yeni R²: 0.5073 (Artış: 0.0040)
'NumberOfBathrooms' çıkarıldı. Yeni R²: 0.5209 (Artış: 0.0136)
'UsingStatus_Boş' çıkarıldı. Yeni R²: 0.5224 (Artış: 0.0015)
'UsingStatus_Kiracı Oturuyor' çıkarıldı. Yeni R²: 0.5232 (Artış: 0.0008)
'BuildStatus_Yapım Aşamasında' çıkarıldı. Yeni R²: 0.5232 (Artış: 0.0000)
'BuildStatus_İkinci El' çıkarıldı. Yeni R²: 0.5232 (Artış: 0.0000)
'NumberOfWCs_2' çıkarıldı. Yeni R²: 0.5233 (Artış: 0.0000)
'NumberOfWCs_4' çıkarıldı. Yeni R²: 0.5234 (Artış: 0.0001)
'NumberOfWCs_6+' çıkarıldı. Yeni R²: 0.5235 (Artış: 0.0001)
'HeatingType_Merkezi (Pay Ölçer)' çıkarıldı. Yeni R²: 0.5240 (Artış: 0.0005)
'HeatingType_Merkezi Doğalgaz' çıkarıldı. Yeni R²: 0.5252 (Artış: 0.0011)
'HeatingType_Merkezi Fueloil' çıkarıldı. Yeni R²: 0.5252 (Artış: 0.0001)
'HeatingType_Yerden Isıtma' çıkarıldı. Yeni R²: 0.5253 (Artış: 0.0001)
'Ad_TotalGu

In [514]:
importances_all_fe_model = tree_reg_fe.feature_importances_
feature_names_all_fe_model = X_train_fe.columns
all_feature_importance_df = pd.DataFrame({'feature': feature_names_all_fe_model, 'importance': importances_all_fe_model})
all_feature_importance_df = all_feature_importance_df.sort_values(by='importance', ascending=False)

top_6_features = all_feature_importance_df.head(6)['feature'].tolist()
print(f"Kullanılacak en iyi 6 özellik: {top_6_features}")

X_train_top6 = X_train_fe[top_6_features]
X_test_top6 = X_test_fe[top_6_features]

tree_reg_top8 = DecisionTreeRegressor(**best_tree_params, random_state=42)
tree_reg_top8.fit(X_train_top6, y_train)
y_pred_top8 = tree_reg_top8.predict(X_test_top6)
mse_top6 = mean_squared_error(y_test, y_pred_top8)
r2_top6 = r2_score(y_test, y_pred_top8)
print(f"En İyi 6 Özellikli Karar Ağacı Modeli - MSE: {mse_top6:.4f}, R²: {r2_top6:.4f}")

# Özellik mühendisliği ve seçimi sonrası elde edilen en iyi 6 özelliği kullanarak modelimizi oluşturduk.

Kullanılacak en iyi 6 özellik: ['NetSquareMeters', 'district_encoded', 'GrossSquareMeters', 'HeatingType_Kombi Doğalgaz', 'NumberFloorsofBuilding', 'NumberOfBathrooms']
En İyi 6 Özellikli Karar Ağacı Modeli - MSE: 0.6793, R²: 0.5360


In [515]:
rf_reg = RandomForestRegressor(n_estimators=200, random_state=42, n_jobs=-1, max_depth=best_tree_params.get('max_depth', 10), min_samples_split=best_tree_params.get('min_samples_split',2), min_samples_leaf=best_tree_params.get('min_samples_leaf',1))
rf_reg.fit(X_train_fe, y_train) 
y_pred_rf = rf_reg.predict(X_test_fe) 
mse_rf = mean_squared_error(y_test, y_pred_rf)
r2_rf = r2_score(y_test, y_pred_rf)
print(f"Random Forest- MSE: {mse_rf:.4f}, R²: {r2_rf:.4f}")
# Özellik mühendisliği ve seçimi sonrası elde edilen en iyi özellikleri kullanarak Random Forest modelimizi oluşturduk.

xgb_reg = XGBRegressor(n_estimators=200, random_state=42, learning_rate=0.1, 
                    max_depth=best_tree_params.get('max_depth',5)//2+1,
                    early_stopping_rounds=10) 
xgb_reg.fit(X_train_fe, y_train, 
            eval_set=[(X_test_fe, y_test)], 
            verbose=False) 
y_pred_xgb = xgb_reg.predict(X_test_fe) 
mse_xgb = mean_squared_error(y_test, y_pred_xgb)
r2_xgb = r2_score(y_test, y_pred_xgb)
print(f"XGBoost - MSE: {mse_xgb:.4f}, R²: {r2_xgb:.4f}")
# XGBoost modelimizi oluşturduk.

Random Forest- MSE: 0.6175, R²: 0.5782
XGBoost - MSE: 0.5679, R²: 0.6121


In [516]:
xgb_reg_top6_model = XGBRegressor(n_estimators=200, random_state=42, learning_rate=0.1, 
                                 max_depth=best_tree_params.get('max_depth',5)//2+1, 
                                 early_stopping_rounds=10) 
xgb_reg_top6_model.fit(X_train_top6, y_train, 
                       eval_set=[(X_test_top6, y_test)], 
                       verbose=False)
y_pred_xgb_top6 = xgb_reg_top6_model.predict(X_test_top6) 
mse_xgb_top6 = mean_squared_error(y_test, y_pred_xgb_top6)
r2_xgb_top6 = r2_score(y_test, y_pred_xgb_top6)
print(f"XGBoost - MSE: {mse_xgb_top6:.4f}, R²: {r2_xgb_top6:.4f}")
# XGBoost modelimizi en iyi 6 özellik ile yeniden oluşturduk.

XGBoost - MSE: 0.6029, R²: 0.5882


En kritik uygulama, price sütununa logaritmik dönüşüm yapmak. Logaritmik dönüşüm yaptıktan sonra negatiflerde olan modelimiz 0.17 değerine kadar çıktı. Hiperparametre optimizasyonu ile daha da 0.50 değerine kadar arttırdık.
Sonrasında Feature engineering kısmı 0.53 değerine kadar arttırdı.
Farklı model olarak Random Forest ve XGBoost denendi ve 0.60 model performansımız değerine kadar çıktı.
Ayrıca En iyi özellik seçimi, decision tree modelimizin performansını arttırmıştı, XGBoost modelimizin performansını da arttırır mı diye denendi ama maalesef modelimizin performansını düşürdü.